# Training

## Dependencies and imports

In [2]:
import pandas as pd

from datasets import Dataset

from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM
from transformers import Seq2SeqTrainer
from transformers import DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainingArguments

## Load pretrained model
[umT5 small](https://huggingface.co/google/umt5-small)

In [4]:
tokenizer = AutoTokenizer.from_pretrained("google/umt5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("google/umt5-small")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/6.84k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.55M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.9M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/6.62k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/771 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.23G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

## Load training and validation datasets

In [5]:
train_dataset = pd.read_csv('../dataset/train.tsv', sep='\t', encoding='utf-8').dropna()
print(train_dataset.shape)
train_dataset.head(100)

(14679, 6)


,topic,region,document_type,document_ref,original_text,oracle_text
0,healthcare,lazio,accreditation,d4b9addca250f5eac44d42a131c67f29,le attività di valutazione e di presa in caric...,Le attività per valutare e prendersi cura dell...
1,healthcare,lombardia,accreditation,3e0abd25f3bcf417e9e0a8b74e158ef5,"Il richiedente non è tenuto a produrre, se non...",Il richiedente non deve presentare i documenti...
2,healthcare,molise,accreditation,1ec2152018d1d765693e850838db5123,Sulla base delle analisi effettuate e delle cr...,Dopo aver esaminato i risultati delle analisi ...
3,healthcare,molise,accreditation,e39345e2a01bddc686fce18ee7e59b63,L'organizzazione ha definito e approvato le li...,L'organizzazione ha stabilito e approvato le r...
4,public_services,molise,service_charter,93860010e913bb4416120f846e4bf50f,A cittadini che siano in possesso o di uno sfr...,A persone che hanno ricevuto un ordine del tri...
...,...,...,...,...,...,...
95,public_services,toscana,public_holdings_rationalization,e7a53fe1397609fa9098bc14713c7df7,Viene di seguito riportata la scheda di rileva...,Ti invio la scheda per la revisione periodica ...
96,public_services,toscana,public_holdings_rationalization,2c2c05f73b28b3d3ec4870436510242f,La società ha svolto attività di trasporto pub...,La società ha gestito i trasporti pubblici loc...
97,public_services,toscana,public_holdings_rationalization,fb9193c9b86df02c0e0f8a2eca9e1efd,alla luce di tutto quanto sopra esposto in mer...,Considerando tutto ciò che è stato detto finor...
98,public_services,basilicata,service_charter,9505701ca5370bb3f3e83d38bdd10369,Il Gestore si impegna ad erogare il servizio i...,Il Gestore si impegna a fornire il servizio in...


In [6]:
val_dataset = pd.read_csv('../dataset/val.tsv', sep='\t', encoding='utf-8').dropna()
print(val_dataset.shape)
val_dataset.head(10)

(1835, 6)


,topic,region,document_type,document_ref,original_text,oracle_text
0,garbage,basilicata,service_charter,a958e53905ca50a2b93e2a1725e08fec,Il gestore garantisce ai cittadini la massima ...,Il gestore si impegna a fornire tutte le infor...
1,garbage,molise,service_charter,b4dbb1567b36e1c2eb401de156a05585,"Il Comune di Campomarino, tramite l'Unione dei...",Il Comune di Campomarino ha assegnato il servi...
2,public_services,basilicata,service_charter,eddcf692d53988aa0cb265d82fba3dda,È possibile richiedere un contributo per l'acq...,Puoi chiedere un aiuto per comprare computer a...
3,healthcare,molise,accreditation,d7a93a47f4f2de7d3e3700d0344f9b6c,"L'organizzazione ha definito, formalizzato e d...","L'organizzazione ha stabilito, scritto e divul..."
4,public_services,molise,service_charter,0b4b1040778604722a8174dcf03da1e7,Un incaricato del Comune si recherà poi al dom...,Un dipendente del Comune andrà a casa della pe...
5,healthcare,campania,planning_acts,9d5302d8ba446dae30c5130cc23b6d6a,Ferma restando la scelta regionale di privileg...,Pur continuando a dare la priorità agli anzian...
6,public_services,basilicata,public_holdings_rationalization,6fd637aef58163b5baaab03f092b6eab,Pratica affidata all'Ufficio legale dell'Ente ...,Richiesta all'ufficio legale dell'Ente per ott...
7,public_services,veneto,service_charter,65f5268a4bc72113ccfac6b67694f5cb,Il Gestore s'impegna a fornire risposta scritt...,Il Gestore si impegna a rispondere per iscritt...
8,public_services,molise,service_charter,93860010e913bb4416120f846e4bf50f,Il Comune è l'istituzione più vicina ai cittad...,Il Comune è l'ente più vicino ai cittadini e d...
9,public_services,molise,service_charter,71eccb42080fe5c8f27e814bfad5a0c6,Al fine di verificare la qualità del servizio ...,Per controllare la qualità del servizio offert...


In [7]:
raw_train_datasets = Dataset.from_dict({'X': train_dataset['original_text'], 'y': train_dataset['oracle_text']})
raw_val_datasets = Dataset.from_dict({'X': val_dataset['original_text'], 'y': val_dataset['oracle_text']})

## Tokenize

In [8]:
PREFIX = "semplifica: "
TOKENIZER_MAX_LENGHT = 1024

def dataset_preprocess(examples):
    inputs = [PREFIX + doc for doc in examples['X']]
    model_inputs = tokenizer(inputs, max_length=TOKENIZER_MAX_LENGHT, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['y'], max_length=TOKENIZER_MAX_LENGHT, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [9]:
tokenized_train_dataset = raw_train_datasets.map(dataset_preprocess, batched=True)
tokenized_val_dataset = raw_val_datasets.map(dataset_preprocess, batched=True)

Map:   0%|          | 0/14679 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1835 [00:00<?, ? examples/s]

## Start finetuning trainig

In [10]:
BATCH_SIZE = 4
EPOCHS = 10

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

training_args = Seq2SeqTrainingArguments(
    output_dir=f"./epochs",
    # batch_size
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    # epochs
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=EPOCHS,
    num_train_epochs=EPOCHS,
    # learning_rate
    warmup_ratio=0.05,
    learning_rate=1e-4,
    lr_scheduler_type="cosine",
    weight_decay=0.01,
    # others
    predict_with_generate=True,
    fp16=False,
    push_to_hub=False,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    data_collator=data_collator,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
)

In [11]:
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,2.642200,1.879170
2,2.243600,1.669045
3,2.010500,1.581219
4,1.886200,1.524396
5,1.807500,1.485992
6,1.740200,1.456571
7,1.673100,1.435692
8,1.654100,1.426432
9,1.632500,1.421621
10,1.632400,1.421705


TrainOutput(global_step=36700, training_loss=2.128624012294842, metrics={'train_runtime': 11822.4479, 'train_samples_per_second': 12.416, 'train_steps_per_second': 3.104, 'total_flos': 1.923773889294336e+16, 'train_loss': 2.128624012294842, 'epoch': 10.0})